Loading in packages

In [1]:
## Standard libraries
import os
import json
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

/tmp/ipython-input-2913985826.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
INFO:lightning_fabric.utilities.seed:Seed set to 42


cuda:0


In [2]:
# torch geometric
try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

Looking in links: https://pytorch-geometric.com/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp312-cp312-linux_x86_64.whl size=3857021 sha256=37ff403be75938a8e4469ca39b6f5d5e9736b5c35ba1efc08d493afb5a89d484
  Stored in directory: /root/.cache/pip/wheels/84/20/50/44800723f57cd798630e77b3ec83bc80bd26a1e3dc3a672ef5
Successfully built torch-scatter
Looking in links: https://pytorch-geometric.com/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp312-cp312-linux_x86_64.whl size=3039806 sha256=fa3d64d1ebeefbc1a88d832444e4f44faf1349f35143b6b415696ae7502a397c
  Stored in directory: /root/.cache/pip/wheels/71/fa/21/bd1d78ce1629aec4ecc924a63b82f6949dda484b6321eac6f2
Succ

# **differnet layer types **

In [3]:
gnn_layer_by_name = {
    "GCN": geom_nn.GCNConv,
    "GAT": geom_nn.GATConv,
    "GraphConv": geom_nn.GraphConv
}

In [5]:
class GNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, num_layers=2, layer_name="GCN", dp_rate=0.1, **kwargs):
        super().__init__()
        gnn_layer = gnn_layer_by_name[layer_name]

        layers = []
        in_channels, out_channels = c_in, c_hidden
        for _ in range(num_layers-1):
            layers += [
                gnn_layer(in_channels=in_channels,
                          out_channels=out_channels,
                          **kwargs),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden
        layers += [gnn_layer(in_channels=in_channels,
                             out_channels=c_out,
                             **kwargs)]
        self.layers = nn.ModuleList(layers)

    def forward(self, x, edge_index):
        for l in self.layers:
            if isinstance(l, geom_nn.MessagePassing):
                x = l(x, edge_index)
            else:
                x = l(x)
        return x

#############################################
#  toy graph
#############################################

# 4 nodes, with simple edges
edge_index = torch.tensor([
    [0, 1, 2, 3, 0, 2],   # sources
    [1, 0, 3, 2, 2, 0]    # targets (undirected)
], dtype=torch.long)

# Random features: 4 nodes × 3 features
x = torch.randn(4, 3)

# Two classes (0 or 1)
y = torch.tensor([0, 1, 0, 1], dtype=torch.long)

# Train on all nodes (toy example)
train_mask = torch.tensor([True, True, True, True])

data = geom_data.Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)

#############################################
# Instantiate model, loss, optimizer
#############################################
model = GNNModel(
    c_in=3,
    c_hidden=8,
    c_out=2,        # two classes
    num_layers=2,
    layer_name="GCN",
    dp_rate=0.1
)

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

#############################################
# Training loop
#############################################
print("Training...")
for epoch in range(200):
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)

    # Only compute loss on train_mask
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        pred = out.argmax(dim=1)
        correct = int((pred == data.y).sum())
        acc = correct / len(data.y)
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.2f}")

#############################################
# Final predictions
#############################################
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
print("\nFinal predictions:", pred.tolist())
print("True labels:       ", data.y.tolist())


Training...
Epoch 000, Loss: 0.7092, Acc: 0.25
Epoch 020, Loss: 0.6806, Acc: 0.50
Epoch 040, Loss: 0.6519, Acc: 0.50
Epoch 060, Loss: 0.6325, Acc: 0.75
Epoch 080, Loss: 0.6403, Acc: 0.75
Epoch 100, Loss: 0.6010, Acc: 0.50
Epoch 120, Loss: 0.6081, Acc: 0.75
Epoch 140, Loss: 0.6093, Acc: 1.00
Epoch 160, Loss: 0.5946, Acc: 0.50
Epoch 180, Loss: 0.5653, Acc: 0.50

Final predictions: [0, 1, 0, 1]
True labels:        [0, 1, 0, 1]


getting the weights to analyze

In [7]:
state = model.state_dict()
print(state.keys()) ## lists all weight matrices and biases

odict_keys(['layers.0.bias', 'layers.0.lin.weight', 'layers.3.bias', 'layers.3.lin.weight'])
